In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
import nibabel as nib
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

model_file = 'BLAHFCN_A2D_SH3000_saved_model_split'
dirs = ['H','L','M','N','O']



In [2]:
class SHDataSet(Dataset):
    def __init__(self,X):
        
        self.X = X
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        
        vec_a = np.reshape(self.X[i,:],(1, 15, 1))
        a = self.to_tensor(vec_a)
    
        return a

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1 * 15, 400)
        self.fc2 = nn.Linear(400,200)
        self.fc3 = nn.Linear(200,50)
        self.fc4 = nn.Linear(50,200)
        self.fc5 = nn.Linear(200,15)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
        
net = Net()
print(net) 

Net(
  (fc1): Linear(in_features=15, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=15, bias=True)
)


In [4]:
model = net.load_state_dict(torch.load(model_file))

In [5]:
for i in range(len(dirs)):
    
    apply_to_file = "/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/%s/prisma/st/A2D_norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz" % (dirs[i])
    save_file = "BLAHFCN_s%s_A2D_SH3000.nii.gz" % (dirs[i])
    print(apply_to_file)
    print(save_file)

    # load and reshape
    nifti_path = os.path.normpath(apply_to_file)
    img = nib.load(nifti_path)
    data = img.get_fdata()
    dims = data.shape
    print(dims)
    X = np.reshape(data,(dims[0]*dims[1]*dims[2],dims[3]))
    print(X.shape)
    print(X.dtype)
    X.astype(float)
    print(X.dtype)
    
    shset = SHDataSet(X)
    print(len(shset))
    
    Y = np.zeros((dims[0],dims[1],dims[2],dims[3]))
    print(Y.shape)
    Y = np.reshape(Y,(dims[0]*dims[1]*dims[2],dims[3]))
    print(Y.shape)
             
    batch_idx = 0
    for batch_idx, (data) in enumerate(shset):
    
        data = Variable(data).float()
    
        net_out = net(data)
        b = net_out.detach().numpy()
        c = np.reshape(b,(1,15))
        #print(c.shape)
        Y[batch_idx,:]=c;
    
    
        if batch_idx % 10000 == 0:
            print(batch_idx)
            #print(b)
    
    Y = np.reshape(Y,(dims[0],dims[1],dims[2],dims[3]))
    print(Y.shape)
    
    new_img = nib.Nifti1Image(Y,img.affine,img.header)
    nib.save(new_img,save_file)

/Users/kurtschilling/Data/harmonization/CDMRI_Challenge_2018/Testing_Data/H/prisma/st/A2D_norm_dwi_SHfitOrder4_EvenOdd2_SH3000.nii.gz
BLAHFCN_sH_A2D_SH3000.nii.gz
(188, 188, 120, 15)
(4241280, 15)
float64
float64
4241280
(188, 188, 120, 15)
(4241280, 15)
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
107

1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390000
1400000
1410000
1420000
1430000
1440000
1450000
1460000
1470000
1480000
1490000
1500000
1510000
1520000
1530000
1540000
1550000
1560000
1570000
1580000
1590000
1600000
1610000
1620000
1630000
1640000
1650000
1660000
1670000
1680000
1690000
1700000
1710000
1720000
1730000
1740000
1750000
1760000
1770000
1780000
1790000
1800000
1810000
1820000
1830000
1840000
1850000
1860000
1870000
1880000
1890000
1900000
1910000
1920000
1930000
1940000
1950000
1960000
1970000
1980000
1990000
2000000
2010000
2020000
2030000
2040000
2050000
2060000
2070000
2080000
2090000
2100000
2110000
2120000
2130000
2140000
2150000
2160000
2170000
2180000
2190000
2200000
2210000
2220000
2230000
2240000
2250000
2260000
2270000
2280000
2290000
2300000
2310000
2320000
2330000
2340000
2350000
2360000
2370000
2380000
2390000
2400000
2410000


2510000
2520000
2530000
2540000
2550000
2560000
2570000
2580000
2590000
2600000
2610000
2620000
2630000
2640000
2650000
2660000
2670000
2680000
2690000
2700000
2710000
2720000
2730000
2740000
2750000
2760000
2770000
2780000
2790000
2800000
2810000
2820000
2830000
2840000
2850000
2860000
2870000
2880000
2890000
2900000
2910000
2920000
2930000
2940000
2950000
2960000
2970000
2980000
2990000
3000000
3010000
3020000
3030000
3040000
3050000
3060000
3070000
3080000
3090000
3100000
3110000
3120000
3130000
3140000
3150000
3160000
3170000
3180000
3190000
3200000
3210000
3220000
3230000
3240000
3250000
3260000
3270000
3280000
3290000
3300000
3310000
3320000
3330000
3340000
3350000
3360000
3370000
3380000
3390000
3400000
3410000
3420000
3430000
3440000
3450000
3460000
3470000
3480000
3490000
3500000
3510000
3520000
3530000
3540000
3550000
3560000
3570000
3580000
3590000
3600000
3610000
3620000
3630000
3640000
3650000
3660000
3670000
3680000
3690000
3700000
3710000
3720000
3730000
3740000
3750000
